# CIFAR-10 Deep Learning with MLOps\n\n**Problem**: Classify images into 10 CIFAR-10 classes.\n\n**Assumptions**: RGB 32×32 inputs; single-label classification; CPU training.\n\n**Limitations**: Small CNN baseline; few epochs; accuracy is modest by design.\n\n**Dataset**: CIFAR-10 (60k images: 50k train, 10k test). Standard normalization used.\n

## Pipeline Overview\n- **Model**: `SmallCNN` (see `src/model.py`).\n- **Preprocessing**: Resize(32,32) → ToTensor → Normalize(mean=(0.4914,0.4822,0.4465), std=(0.247,0.243,0.261)).\n- **Training & Tracking**: `python -m src.train` logs metrics/artifacts to **MLflow** (`./mlflow_tracking`).\n- **Evaluation**: `python -m src.evaluate` prints `TEST_ACC` and logs to MLflow.\n- **Versioning**: Code via **Git/GitHub**; data/artifacts via **DVC** (`data.dvc`, `dvc.yaml`, `dvc.lock`).\n- **CI**: GitHub Actions runs `pytest` on push.\n- **Serving**: **FastAPI** (`src/predict_service.py`) — `/predict` for inference, `/health` for health.\n- **Container**: `Dockerfile` → `docker build -t cifar-service .` then `docker run -p 8000:8000 cifar-service`.\n- **Monitoring**: Each request logs channel mean/std and simple drift flags to `logs/service.log`.\n

In [ ]:
# Show test accuracy by calling the existing evaluator\nimport subprocess, sys\ncompleted = subprocess.run([sys.executable, '-m', 'src.evaluate'], capture_output=True, text=True)\nprint(completed.stdout.strip() or completed.stderr.strip())\n

## Evidence (CI / Tracking / Serving / Monitoring)\n\nAdd your screenshots here (paths relative to repo). If your filenames contain spaces, URL-encode them or rename. Examples:\n\n- **MLflow experiments / run detail**\n  \n  `![](screens/mlflow_runs.png)`\n\n- **Swagger UI**\n  \n  `![](screens/swagger.png)`\n\n- **/predict result**\n  \n  `![](screens/predict.png)`\n\n- **Monitoring log**\n  \n  `![](screens/monitoring_log.png)`\n\nIf your actual screenshot filenames differ, replace the names above with yours (they are already committed under `screens/`).\n

## Observations\n- The baseline CNN achieves the reported **TEST_ACC** above (printed by the code cell).\n- MLflow records params/metrics and the saved `model.pt` artifact.\n- CI confirms tests pass (`tests/test_model.py`).\n- The API `/predict` works and logs basic **drift flags** if channel stats deviate (>15%).\n- DVC tracks dataset and model artifact versions; `dvc repro` can rerun the pipeline.\n